In [3]:
import datasets

dataset = datasets.load_dataset(r'istvoices_text_dataset.py', cache_dir=r'C:\Users\Habram\.cache')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset istvoices_text_dataset downloaded and prepared to C:/Users/Habram/.cache/istvoices_text_dataset/default/0.0.0/b44092c7d124c65f49aa3d74b95a0b18b3b9096334970b6506fbc4bf1b03494f. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]